<a href="https://colab.research.google.com/github/DonghaeSuh/kw-AI-hackathon/blob/main/kfold_data_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/test_data')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from tqdm import tqdm
from numpy.fft import fft, fftshift
import matplotlib.pyplot as plt
import seaborn as sns
import random
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import os
from sklearn.preprocessing import StandardScaler
from keras import backend as K

In [ ]:
train=pd.read_csv('train_features.csv')
train_label=pd.read_csv('train_labels.csv')
test=pd.read_csv('test_features.csv')
sub=pd.read_csv('sample_submission.csv')

In [ ]:
train['acc_Energy']=(train['acc_x']**2+train['acc_y']**2+train['acc_z']**2)**(1/3)
test['acc_Energy']=(test['acc_x']**2+test['acc_y']**2+test['acc_z']**2)**(1/3)

train['gy_Energy']=(train['gy_x']**2+train['gy_y']**2+train['gy_z']**2)**(1/3)
test['gy_Energy']=(test['gy_x']**2+test['gy_y']**2+test['gy_z']**2)**(1/3)

train['gy_acc_Energy']=((train['gy_x']-train['acc_x'])**2+(train['gy_y']-train['acc_y'])**2+(train['gy_z']-train['acc_z'])**2)**(1/3)
test['gy_acc_Energy']=((test['gy_x']-test['acc_x'])**2+(test['gy_y']-test['acc_y'])**2+(test['gy_z']-test['acc_z'])**2)**(1/3)

train["acc_total"]=(train["acc_x"]**2+train["acc_y"]**2+train["acc_z"]**2)**0.5
test["acc_total"]=(test["acc_x"]**2+test["acc_y"]**2+test["acc_z"]**2)**0.5

In [ ]:
dt=0.02 
def jerk_signal(signal): 
        return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])

In [ ]:
train_dt=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    train_dt.append(temp)

100%|██████████| 3125/3125 [01:06<00:00, 46.79it/s]


In [ ]:
test_dt=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    test_dt.append(temp)

100%|██████████| 782/782 [00:14<00:00, 54.15it/s]


In [ ]:
from scipy import fftpack
from numpy.fft import *

def fourier_transform_one_signal(t_signal):
    complex_f_signal= fftpack.fft(t_signal)
    amplitude_f_signal=np.abs(complex_f_signal)
    return amplitude_f_signal

In [ ]:
train=pd.concat(train_dt)

In [ ]:
fft=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for i in train.columns[2:]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft.append(temp)
train=pd.concat(fft)

100%|██████████| 3125/3125 [00:25<00:00, 124.82it/s]


In [ ]:
test=pd.concat(test_dt)

In [ ]:
fft_t=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for i in test.columns[2:]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft_t.append(temp)
test=pd.concat(fft_t)

100%|██████████| 782/782 [00:04<00:00, 179.71it/s]


In [ ]:
col=train.columns
train_s=train.copy()
test_s=test.copy()

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

scaler = StandardScaler()

train_s.iloc[:,2:]= scaler.fit_transform(train_s.iloc[:,2:])
train_sc = pd.DataFrame(data = train_s,columns =col)

test_s.iloc[:,2:]= scaler.transform(test_s.iloc[:,2:])
test_sc = pd.DataFrame(data = test_s,columns =col)

In [ ]:
def make_dataset(data):
  ids = data['id'].unique()
  id_data = data.groupby('id')
  series_data = []

  for i in ids:
    df = id_data.get_group(i)
    df = df.drop(['id', 'time'], axis=1)
    series_data.append(df.to_numpy())

  series_data = np.array(series_data)
  return series_data

In [ ]:
series_train = make_dataset(train_sc)
series_test = make_dataset(test_sc)

In [ ]:
series_train.shape, series_test.shape

((3125, 600, 20), (782, 600, 20))

In [ ]:
labels = train_label['label'].to_numpy()

In [ ]:
import tensorflow as tf

In [ ]:
def make_train(series_data, labels):
  cat_y = tf.keras.utils.to_categorical(labels)

  BATCH_SIZE = 128
  train_dataset = tf.data.Dataset.from_tensor_slices((series_data, cat_y))
  train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000, seed=42)
  train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  return train_dataset

def make_val(series_data, labels):
  cat_y = tf.keras.utils.to_categorical(labels)

  BATCH_SIZE = 128
  val_dataset = tf.data.Dataset.from_tensor_slices((series_data, cat_y))
  val_dataset = val_dataset.batch(BATCH_SIZE)
  val_dataset = val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  return train_dataset

In [ ]:
from tensorflow import keras

In [ ]:
def weighted_loss(labels, y_pred, beta=0.9):
        no_samples = [12, 21, 20, 23, 35, 25, 24, 26, 97, 37, 20, 23, 12,
                     12, 25, 25, 22, 27, 47, 20, 26, 27, 19, 20, 35, 24,
                     1518, 34, 55, 20, 35, 20, 18, 20, 22, 30, 28, 35, 20,
                     20, 34, 20, 20, 35, 21, 22, 20, 26, 25, 30, 37, 24,
                     12, 13, 23, 37, 36, 20, 20, 23, 48]

        weight = tf.divide(1-beta, 1-tf.pow(beta, no_samples))
        #y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        loss = labels * K.log(y_pred) * weight
        loss = -K.sum(loss, -1)
        return loss

# 모델을 만들어 주는 함수
# 기존 base에서 overfitting이 심해, dropout을 늘림(아직 제출은 안해봄)
def base():
 model = keras.models.Sequential([
            keras.layers.Conv1D(128, 9, padding='same', input_shape=[600, 20]),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.3),
            keras.layers.Conv1D(256, 6, padding='same'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.4),
            keras.layers.Conv1D(128, 3,padding='same'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.5),
            keras.layers.GlobalAveragePooling1D(),
            keras.layers.Dense(61, activation='softmax')
  ])
 model.compile(optimizer='adam',
                loss='categorical_crossentropy', 
                metrics=['accuracy'])
 return model

In [ ]:
# checkpoint path
# 중간중간 모델의 weight를 저장할 경로 설정
ckpt_name = 'k_fold_cnn_ckpt.hdf5'
checkpoint_dir_path = 'checkpoint'
checkpoint_path = os.path.join('checkpoint', ckpt_name)

# check checkpoint paht
# 경로가 없으면 생성함
if not(os.path.exists(checkpoint_dir_path)):
  os.mkdir(checkpoint_dir_path)

# callback 함수 목록
callbacks_list = [
    # 매 epoch 마다 val_loss를 체크하여 가장 낮은 상태의 weight를 저장
    tf.keras.callbacks.ModelCheckpoint(
        filepath = checkpoint_path,
        monitor='val_loss',
        mode='min',
        save_weights_only=True,
        save_best_only=True
    ),
    # 8번 동안 val_loss의 향상이 없으면 훈련 종료
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        mode='min',
        verbose=1, 
        patience=8
    )
]

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
# k-fold 적용
# 데이터 수가 적어 10개의 fold 사용
k = 10 
split = StratifiedKFold(n_splits=k, random_state=42, shuffle=True)
results = []    # 매 fold 훈련 후 loss 저장
models = []     # 매 fold 훈련 후 해당 모델 저장

for i, (train, val) in enumerate(split.split(series_train, labels)):
  print('Fold', i)
  print('#'*20)

  train_dataset = make_train(series_train[train], labels[train])
  val_dataset = make_val(series_train[val], labels[val])

  model = base()

  model.fit(train_dataset, validation_data=val_dataset, callbacks=callbacks_list, epochs=50)

  # 모델 복원
  model.load_weights(checkpoint_path)
  result = model.evaluate(val_dataset)[0]

  results.append(result)
  models.append(model)

  del model, train_dataset, val_dataset

Fold 0
####################
Epoch 1/50
22/22 [==============================] - 107s 5s/step - loss: 2.9014 - accuracy: 0.4417 - val_loss: 5.2570 - val_accuracy: 0.0480
Epoch 2/50
22/22 [==============================] - 105s 5s/step - loss: 2.1783 - accuracy: 0.5260 - val_loss: 3.0350 - val_accuracy: 0.3766
Epoch 3/50
22/22 [==============================] - 107s 5s/step - loss: 1.9837 - accuracy: 0.5484 - val_loss: 2.1262 - val_accuracy: 0.5295
Epoch 4/50
22/22 [==============================] - 103s 5s/step - loss: 1.8644 - accuracy: 0.5537 - val_loss: 1.9564 - val_accuracy: 0.5473
Epoch 5/50
22/22 [==============================] - 105s 5s/step - loss: 1.7489 - accuracy: 0.5761 - val_loss: 1.8356 - val_accuracy: 0.5519
Epoch 6/50
22/22 [==============================] - 104s 5s/step - loss: 1.6628 - accuracy: 0.5829 - val_loss: 1.8790 - val_accuracy: 0.5192
Epoch 7/50
22/22 [==============================] - 103s 5s/step - loss: 1.5833 - accuracy: 0.5964 - val_loss: 1.9887 - val_ac

In [ ]:
# 결과 생성
pred_list = []    # 예측 결과를 담을 리스트
for model in models:
  pred = model.predict(series_test)
  pred_list.append(pred)

pred = np.mean(pred_list, axis=0)

In [ ]:
# 제출물 생성
sub.iloc[:, 1:] = pred
sub.to_csv('submission_3.csv', index=False)